In [1]:
import numpy as np
import png
import pydicom
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from os import listdir
from os.path import isfile, join
import os
import torch
import torch.nn as nn
from torchvision.transforms import Compose
import transform_classes

from roi import RoiLearn
from roi_dataset import RoiDataset
from preprocessor import Preprocessor

import preprocess_img

In [61]:
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, n_inp, n_hidden):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(n_inp, n_hidden)
        self.decoder = nn.Linear(n_hidden, n_inp)
        self.n_inp = n_inp
        self.n_hidden = n_hidden

    def forward(self, x):
        encoded = F.sigmoid(self.encoder(x))
        decoded = F.sigmoid(self.decoder(encoded))
        return encoded, decoded

def kl_divergence(p, q):
    '''
    args:
        2 tensors `p` and `q`
    returns:
        kl divergence between the softmax of `p` and `q`
    '''
    p = F.softmax(p)
    q = F.softmax(q)

    s1 = torch.sum(p * torch.log(p / q))
    s2 = torch.sum((1 - p) * torch.log((1 - p) / (1 - q)))
    return s1 + s2

In [66]:
import numpy as np
import png
import pydicom
from sklearn.preprocessing import normalize

from os import listdir
from os.path import isfile, join
import os
import torch
import torch.nn as nn


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class RoiLearn:
    def __init__(self):
        torch.manual_seed(12)
        self.conv1 = nn.Conv2d(1,100, (11,11))
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.avgpool = nn.AvgPool2d(6)
        self.flatten = Flatten()
        self.full = nn.Linear(8100,1024)
        #self.encoder = nn.Linear(121,100)
        #self.decoder = nn.Linear(100,121)
          
    # Autoencoder architecture
    def build_ae(self):
        self.autoencoder = Autoencoder(121,100)
        self.autoencoder = self.autoencoder.double()
        
    # Autoencoder W2 and b2 to the original model conv1 layer features and biases.
    # From the parameters list - index 0 is the weights
    #                          - index 1 is the biases
    def ae_weights2model_feature_set(self):
        
        w2 = list(self.encoder.parameters())
        
        b2 = w2[1].detach().numpy()
        # weights shape here (100,121)
        w2 = np.expand_dims(w2[0].detach().numpy().reshape((100,11,11)), axis = 1)
        # weights shape (100,1,11,11)
        
        conv1_features = list(self.conv1.parameters())
        conv1_features[0] = torch.nn.Parameter(torch.from_numpy(w2))
        conv1_features[1] = torch.nn.Parameter(torch.from_numpy(b2))
        conv1_features[0].requires_grad=False
        conv1_features[1].requires_grad=False
        
    # Rho - sparsity penalty pj = p    
    def learn_ae(self, dataset_loader, optimizer, ep = 1, lr = 0.01, rho = torch.tensor(0.1).double(),BETA = 3, RHO = 0.1):
        rho = torch.FloatTensor([RHO for _ in range(self.autoencoder.n_hidden)]).unsqueeze(0).double()
        for epoch in range(ep):
            for i_batch, sample_batched in enumerate(dataset_loader):
                #print(i_batch, sample_batched['image'].size(),sample_batched['mask'].size())
                #print(sample_batched['image'].shape)
                
                encoded, decoded = self.autoencoder(sample_batched['image'])
                MSE_loss = (sample_batched['image'] - decoded) ** 2
                MSE_loss.view(1, -1).sum(1)
                MSE_loss = MSE_loss.view(1, -1).sum(1) / dataset_loader.batch_size
                
                #y_pred = self.autoencoder(sample_batched['image'])
                rho_hat = torch.sum(encoded, dim=0, keepdim=True)
                sparsity_penalty = BETA * kl_divergence(rho, rho_hat)
                
                loss = MSE_loss + sparsity_penalty
                
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print('epoch: ', epoch,' loss: ', loss.item())
                
    
    def build_model(self):
        self.model = nn.Sequential(self.conv1,
                            self.avgpool,
                            self.softmax,
                            self.flatten,
                            self.full,
                            self.softmax
                            )
        self.model = self.model.double()
    
    def propagate_from_dataLoader(self,dl):
        for i_batch, sample_batched in enumerate(dl):
            print(self.model(sample_batched[0]))
        
    def propagate(self):
        return self.model(self.x)
    
    def save_image( self,npdata, outfilename ) :
        img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), "L" )
        img.save( outfilename )


In [47]:
# if we don't have the .csv file
preprocess_img.write_all_rectangle2file('O:\\ProgrammingSoftwares\\anaconda_projects\\heart_contour\\sa_all_1\\')

Dicom files were read in!
Con files were read in!
Dicom files were read in!
Con files were read in!
Dicom files were read in!
Con files were read in!
Dicom files were read in!
Con files were read in!


In [68]:
csv_file = 'O:/ProgrammingSoftwares/anaconda_projects/heart_contour/sa_all_1/rectangle.csv'

compose3 = Compose([transform_classes.GetRandomPatch(),transform_classes.StandardScale2(),transform_classes.ToTensor()])

ds2 = RoiDataset(csv_file, compose3)
roi = RoiLearn()
roi.build_ae()

crit = torch.nn.MSELoss()
opt = torch.optim.SGD(roi.autoencoder.parameters(), lr=0.001, weight_decay = 0.0001 )

# Random 1000 sample
weighted_rnd_sample = torch.utils.data.WeightedRandomSampler([float(1/len(ds2)) for i in range(len(ds2))], 1000, replacement=True)
dataset_loader = torch.utils.data.DataLoader(ds2,batch_size=8, num_workers=0, sampler=weighted_rnd_sample)

roi.learn_ae(dataset_loader, optimizer = opt,  ep = 10)

#roi.ae_weights2model_feature_set()

epoch:  0  loss:  135.04837830955535
epoch:  1  loss:  128.15998807795094
epoch:  2  loss:  110.37061060556725
epoch:  3  loss:  122.25503991519653
epoch:  4  loss:  117.5149186699118
epoch:  5  loss:  119.85674924642241
epoch:  6  loss:  118.53525961351876
epoch:  7  loss:  116.07255359897056
epoch:  8  loss:  118.26996631444287
epoch:  9  loss:  118.46232447794543


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision.transforms import Compose
import transform_classes

csv_file = 'O:/ProgrammingSoftwares/anaconda_projects/heart_contour/sa_all_1/rectangle.csv'
compose1 = Compose([transform_classes.ReScale64(),transform_classes.StandardScale(),transform_classes.ToTensor()])
compose2 = Compose([transform_classes.ReScale32(),transform_classes.ToTensor()])
ds = RoiDataset(csv_file, compose1, compose2)

roi = RoiLearn()
roi.build_model()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(roi.model.parameters(), lr=0.1)

dataset_loader = torch.utils.data.DataLoader(ds,batch_size=32, shuffle=True,num_workers=0)